In [1]:
from utilities.utils_clean import *
from utilities.test_dataset_utils import *
from utilities.nlp_processing import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test_data = load_from_pickle("../data/test_dataset_2015.pkl")

df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
df_test_data.columns = ["patentnumber","xml"] 


# df = read_xlsb_file()
# df[(df.issueyear == 2015) | (df.prophetic !=0)][["patentnumber","prophetic","nonprophetic","issueyear"]]
# df[df.issueyear == 2015].to_csv("freilichdataet_2015.csv")
df = pd.read_csv("../data/freilichdataet_2015.csv")
df["patentnumber"] = df["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
df["patentnumber"] = df["patentnumber"].apply(remove_leadiong_zeros)
df.head()

Loaded 25081 patents from ../data/test_dataset_2015.pkl


,Unnamed: 0,patentnumber,appyear,issueyear,inventorcountry,divisional,continuation,prioritydate,priority.formatted,priorityyear,...,yr4,yr8,yr11,claims,wordaverage,litigated,orangebook,forward.cites,industry,subindustry
0,497653,8969641,1976.0,2015.0,US,0.0,0.0,19760903.0,28006.0,1976.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0
1,497654,9134302,1995.0,2015.0,US,1.0,1.0,19830505.0,30441.0,1983.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
2,497655,8926966,1995.0,2015.0,US,1.0,1.0,19900921.0,33137.0,1990.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,3.0,31.0
3,497656,9181140,1993.0,2015.0,US,0.0,1.0,19920916.0,33863.0,1992.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
4,497657,9199886,2009.0,2015.0,US,0.0,1.0,19940119.0,34353.0,1994.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [3]:
merged = df_test_data.merge(df, on="patentnumber", how="left")
print(len(merged))
cols = ["patentnumber","xml","prophetic","nonprophetic","allprophetic","someprophetic"]
merged[cols].head()

25081


,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",1.0,12.0,0.0,1.0
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",26.0,0.0,1.0,0.0
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",6.0,5.0,0.0,1.0


In [ ]:
def extract_examples_start_w_word_test(xml_siblings):
    examples = []
    current_example = None
    in_example = False

    for tag in xml_siblings:
        if tag.name == "heading":
            if (
                tag.text.strip().lower().startswith("example")
                or tag.text.strip().lower().startswith("experiment")
                or tag.text.strip().lower().startswith("test")
            ):
                in_example = True
                current_example = {
                    "number": tag.text.strip(),
                    "title": xml_siblings[xml_siblings.index(tag) + 1].text.strip(),
                    "content": [],
                }
                examples.append(current_example)
        elif tag.name == "heading" and (tag.text.strip().lower().startswith("example")
                or tag.text.strip().lower().startswith("experiment")
                or tag.text.strip().lower().startswith("test")):
                in_example = False
            # else:
            #     # If we hit any other heading, stop collecting content
            #     in_example = False
        elif in_example and current_example is not None:
            current_example["content"].append(tag.text.strip())

    return examples



In [113]:
def extract_examples(merged, num_of_patents=1000):
    """
    Analyze patents to extract and compare example counts
    
    Args:
        merged: DataFrame containing patent data
        cols: Columns to use from the DataFrame
        num_of_patents: Number of patents to analyze (default=1000)
    
    Returns:
        dict: Statistics about the analysis results
    """
    mae = 0
    highest_difference = 0
    found_heading = 0
    not_found_heading = 0
    gib = 0
    short = 0
    mostdifss = []
    doc_w_exp = {}
    highest_difference_patent = None
    xmls = []
    for row in merged[:num_of_patents].iterrows():
        xml = row[1]["xml"]
        if len(xml) <= 2000:

            short += 1
            continue  
        s_tags = re.findall(r'<s\d+>.*?</s\d+>', xml)
        if len(s_tags) > 0 or '<sequence-cwu id="SEQLST-0">' in xml:
            gib += 1
            xmls.append(xml)
            continue
            
        heading = extract_experiments_w_heading(xml)
        janetsnumexamples = row[1]["prophetic"] + row[1]["nonprophetic"]

        # Process examples based on heading presence
        if heading and len(heading) == 1:
            found_heading += 1
            examples = extract_examples_start_w_word_test(heading[0].find_next_siblings())
            if len(examples) == 0:
                soup = BeautifulSoup(xml, 'xml')
                siblings = soup.find_all(['heading', 'p'])
                examples = extract_examples_start_w_word_test(siblings)
        else:
            not_found_heading += 1
            soup = BeautifulSoup(xml, 'xml')
            siblings = soup.find_all(['heading', 'p'])
            examples = extract_examples_start_w_word_test(siblings)
        numexamples = len(examples)
        difference = abs(numexamples - janetsnumexamples)
        mae += difference
        
        # if difference > 0:
        #     mostdifss.append([difference, row[1]["patentnumber"]])
        # if difference > highest_difference:
        #     highest_difference = difference
        #     highest_difference_patent = row[1]["patentnumber"]
        # if isinstance(examples, list) and len(examples) > 0:
        #     exs = []
        #     for ex in examples:
        #         if len(ex["content"])>0:
        #             exs.append(ex["content"])
        #     examples = exs
        # elif len(examples) > 0:
        #         doc_w_exp[row[1]["patentnumber"]] = examples
        if difference > 0:
            mostdifss.append([difference, row[1]["patentnumber"]])
        if difference > highest_difference:
            highest_difference = difference
            highest_difference_patent = row[1]["patentnumber"]
        if len(examples) > 0:
            doc_w_exp[row[1]["patentnumber"]] = examples

    results = {
        "found_heading": found_heading,
        "not_found_heading": not_found_heading,
        "gibberish": gib,
        "too_short": short,
        "mae": mae/(found_heading + not_found_heading),
        "total_error": mae,
        "highest_difference": highest_difference,
        "highest_difference_patent": highest_difference_patent,
        "most_differences": mostdifss,
        "documents_with_examples": doc_w_exp,
        "gibs": xmls
    }
    
    return results


num_of_patents = 1000
stats = extract_examples(merged, num_of_patents=num_of_patents)
print(f"found_heading: {stats['found_heading']}, not_found_heading: {stats['not_found_heading']}, gibberish: {stats['gibberish']}, too_short: {stats['too_short']}")
print(f"Num good xmls: {stats['found_heading'] + stats['not_found_heading']}, out of {num_of_patents}")
print(f"Extracted patents with content {len(stats['documents_with_examples'].keys())/(stats['found_heading'] + stats['not_found_heading'])} patents MAE: {stats['mae']}, total_error: {stats['total_error']}, highest_difference: {stats['highest_difference']}, highest_difference_patent: {stats['highest_difference_patent']}")

found_heading: 528, not_found_heading: 272, gibberish: 176, too_short: 24
Num good xmls: 800, out of 1000
Extracted patents with content 0.99875 patents MAE: 0.3375, total_error: 270.0, highest_difference: 56.0, highest_difference_patent: 8933072


In [6]:
stats["documents_with_examples"]["8927121"]

[{'number': 'Example 1',
  'title': 'General Synthetic Methods',
  'content': ['']},
 {'number': 'Example 1.1.1',
  'title': '',
  'content': ['',
   'A mixture of 4′-Bromo-(1,1′-biphenyl)-4-ol (10.0 g, 40.1 mmol), 4-iodoanisole (18.72 g, 80.0 mmol), cesium carbonate (26.1 g, 80.2 mmol), copper iodide (760 mg, 4.0 mmol), dimethylglycine hydrochloride (1.68 g, 12.0 mmol), and anhydrous 1,4-dioxane (50 mL) was purged via freeze-pump-thaw method. The resulting mixture was heated to 110° C. overnight. After cooling, the resulting mixture was poured into ethyl acetate (300 mL), and stirred at 40° C. for 30 min; and the resulting solids were filtered off. The filtrate was dried under vacuum to give ivory solids. The ivory solids were washed with a mixture of ethyl acetate and methanol to give pure product (Compound 1); 6.5 g, 46% yield; confirmed by HNMR']},
 {'number': 'Example 1.1.2',
  'title': '',
  'content': ['',
   'Compound 1 (2 g, 5.63 mmol) was dissolved in anhydrous tetrahydrofura

In [ ]:
# import nltk
# from nltk import pos_tag, word_tokenize
# from bs4 import BeautifulSoup
# import re


# nltk.download("averaged_perceptron_tagger")
# nltk.download("punkt")


# # region tense analysis
# def check_tense_nltk(sentence):
#     words = word_tokenize(sentence)
#     tagged = pos_tag(words)

#     past = ["VBD", "VBN"]
#     present = ["VB", "VBG", "VBP", "VBZ", "MD"]

#     tenses = {"past": 0, "present": 0}

#     for word, tag in tagged:
#         if tag in past:
#             tenses["past"] += 1
#         elif tag in present:
#             tenses["present"] += 1
#         elif word.lower() in ["will", "shall"]:
#             tenses["present"] += 1

#     return max(tenses, key=tenses.get) if max(tenses.values()) > 0 else "Unknown"

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Primary tense: present


In [9]:
"".join((ex[0]["content"]))

'TiO2 particles were first coated with Gantrez S-97BF polymer and then with KF9909 silicone fluid under sonication, and the coated particles were then mixed with more KF9909 silicone fluid in a main beaker and mixed together using a pro mixer manufactured by Caframo in Ontario, Canada (Model # BDC 1850) at digital reading of 105 until a homogenous mixture was formed. Stannous octoate, which was commercially available from Arkema as Fascat® 2003 catalyst, was added into the main beaker at room temperature, and mixing was resumed for about 1 minute. The resulting mixture was immediate poured onto a flat surface and allowed to sit for 30 minutes, thereby forming a cross-linked flat sheet. The flat sheet was subsequently reduced to particles by mortar pestle. The resulting treated particles had an average particle size ranging from about 1 micron to 10 microns, as measured under microscope. Such treated particles contained about 85.31 wt % of TiO2, 3.52% Gantrez S-97BF, 0.41% stannous octo

In [51]:
tokens = word_tokenize("".join((ex[125]["content"])))
tagged = pos_tag(tokens)
d = pd.DataFrame(tagged)
d.columns = ["word","tag"]
d.tag.unique()

array(['NNP', 'NN', '(', 'CD', ':', 'CC', 'IN', ')', 'VBZ', 'VBN', 'JJ',
       '.', 'DT', 'RB', 'NNS', 'VBD', 'TO', 'VBG', ',', 'VB'],
      dtype=object)

In [84]:
merged.patentnumber.head()

0    RE045323
1    RE045324
2    RE045325
3     8925349
4     8925551
Name: patentnumber, dtype: object

In [75]:
import pprint as pp
pp.pprint(merged[merged.patentnumber == "8927557"]["xml"].values[0])

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v45-2014-04-03.dtd" [ ]>\n'
 '<us-patent-grant lang="EN" dtd-version="v4.5 2014-04-03" '
 'file="US08927557-20150106.XML" status="PRODUCTION" id="us-patent-grant" '
 'country="US" date-produced="20141219" date-publ="20150106">\n'
 '<us-bibliographic-data-grant>\n'
 '<publication-reference>\n'
 '<document-id>\n'
 '<country>US</country>\n'
 '<doc-number>08927557</doc-number>\n'
 '<kind>B2</kind>\n'
 '<date>20150106</date>\n'
 '</document-id>\n'
 '</publication-reference>\n'
 '<application-reference appl-type="utility">\n'
 '<document-id>\n'
 '<country>US</country>\n'
 '<doc-number>13390940</doc-number>\n'
 '<date>20100824</date>\n'
 '</document-id>\n'
 '</application-reference>\n'
 '<us-application-series-code>13</us-application-series-code>\n'
 '<priority-claims>\n'
 '<priority-claim sequence="01" kind="regional">\n'
 '<country>EP</country>\n'
 '<doc-number>09168685</doc-number>\n'
 '<date>20

In [10]:
mosftdf = pd.DataFrame(stats["most_differences"])
mosftdf.columns = ["difference","patentnumber"]
mosftdf.sort_values("difference",ascending=False).head(10)

,difference,patentnumber
67,56.0,8933072
36,15.0,8927721
63,12.0,8933042
8,11.0,8926966
35,7.0,8927710
100,6.0,8937055
82,6.0,8933229
4,6.0,8926862
55,6.0,8932717
23,6.0,8927588


In [10]:
dic_to_dic_w_tense_test({"8927557":stats["documents_with_examples"]["8927557"],"8927542" : stats["documents_with_examples"]["8927542"]})

{'8927557': {'past': 1, 'present': 0, 'unknown': 0},
 '8927542': {'past': 0, 'present': 0, 'unknown': 0}}

In [ ]:
# gib_path = "gilberish_xmls"
# os.makedirs(gib_path, exist_ok=True)
# for i, xml in enumerate(stats["gibs"]):
    
#     save_xml_string(xml, f"{gib_path}/gibberish_{find_doc_number(xml)}.xml")

In [104]:
stats["documents_with_examples"]["8925551"]

[{'number': 'Example 1',
  'title': 'Preferred Embodiment of the Invention',
  'content': ['In its preferred embodiment the device is composed by a thin flexible hollow shaft with inflatable balloons as retractor and anchor members (FIG. 5 A). The materials and manufacture of such a device is well known in the art as exemplified by angioplasty catheters, tracheotomy tubes, jejunostomy tubes and other biocompatible medical devices that are temporarily or permanently implanted within body tissue. As representative examples of the art, U.S. Pat. No. 3,659,612 to Shiley describes a cannula that is placed in the trachea with a refillable balloon at its internal end; U.S. Pat. No. 4,335,723 to Patel describes the use of thermoplastic elastomer material for balloon cathers; U.S. Pat. No. 6,013,728 to Chen describes biocompatible polymers used in construction of vascular catheters and expansion balloons, U.S. Pat. No. 4,254,774 to Beritos teaches the manufacture of catheters with external diam

In [ ]:
final = dic_to_dic_w_tense_test(stats["documents_with_examples"],threshold = 0)
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"number of unknown tensed examples: {df_check['unknown'].sum()}")
print(f"Total of examples in freilich {df_check.Total_Freilich.sum()},Total examples extracted {df_check.Total_Extracted.sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)


# Number of exact matches: 386 out of 799 , Percentage: 48.310387984981226
# Number of exact num of patent extracted: 680 out of 799
# Avg Total error: 1.1476846057571966, num of corrects:  680
# Avg Total prophetic error: 1.4292866082603255, num of corrects: 439
# Avg Total nonprophetic error: 2.3642052565707132, num of corrects: 402
# 3948.0
# number of unknown tensed examples: 670
# Total of examples in freilich 11897.0,Total examples extracted 11388


# Number of exact matches: 254 out of 406 , Percentage: 62.5615763546798
# Number of exact num of patent extracted: 359 out of 406
# Avg Total error: 0.2684729064039409, num of corrects:  359
# Avg Total prophetic error: 3.7660098522167487, num of corrects: 286
# Avg Total nonprophetic error: 3.9753694581280787, num of corrects: 256
# 3252.0
# number of unknown tensed examples: 2
# Total of examples in freilich 7238.0,Total examples extracted 7283

#patent 8933072 in janets says 9 nonproph we find 36 present 29past
# 8927721 she doesnt extract experiments or test she only extracts example
#8933042 janet found 1 threres more
#8933072 janet found 9,theremany more
#8926862 janets found 3 theres more

# Number of exact matches: 574 out of 799 , Percentage: 71.8397997496871
# Number of exact num of patent extracted: 699 out of 799
# Avg Total error: 0.3241551939924906, num of corrects:  699
# Avg Total prophetic error: 0.6933667083854819, num of corrects: 640
# Avg Total nonprophetic error: 0.8748435544430538, num of corrects: 592
# 1512.0
# number of unknown tensed examples: 0
# Total of examples in freilich 11897.0,Total examples extracted 12150

Past
Propane-Propene Adsorption Isotherms in CPO-27 (Fe).Propane and propylene adsorption measurements can be performed at thermodynamic equilibrium at 40° C. from 1 g of CPO-27(Fe) solid previously activated at different temperatures (20, 50, 100, 150, 200, 250° C.) under primary vacuum (P=1 Pa for 16 hours)The expected profile of the adsorption isotherms of propane and propylene will be very similar to that obtained for the Ni compound (shown in the previous example), with a higher propane-propylene selectivity at low pressures and larger values at higher pressures. Considering the retrodonation effect of the iron, a better propane-propylene separation is expected.
Past
Gel and Gel Sandwich Natrosol® and like HMHECs of the disclosure can additionally or alternatively be used to encapsulate cells or cell clusters. The HMHEC powder, for example, can be directly dissolved in culture media, autoclaved, and then poured into, or used to coat, the cell culture vessels. Depending on the stre

,patentnumber,present,past,unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
558,8933072,4,61,0,0.0,9.0,0.0,0.0,65,9.0,4.0,52.0,56.0,112.0
298,8927721,9,7,0,0.0,1.0,0.0,0.0,16,1.0,9.0,6.0,15.0,30.0
545,8933042,1,12,0,0.0,1.0,0.0,0.0,13,1.0,1.0,11.0,12.0,24.0
289,8927710,2,569,0,0.0,564.0,0.0,0.0,571,564.0,2.0,5.0,7.0,14.0
647,8933229,0,33,0,0.0,27.0,0.0,0.0,33,27.0,0.0,6.0,6.0,12.0
45,8926862,5,4,0,3.0,0.0,1.0,0.0,9,3.0,2.0,4.0,6.0,12.0
485,8932717,0,9,0,0.0,3.0,0.0,0.0,9,3.0,0.0,6.0,6.0,12.0
227,8927588,4,285,0,0.0,283.0,0.0,0.0,289,283.0,4.0,2.0,6.0,12.0
795,8937055,0,20,0,0.0,14.0,0.0,0.0,20,14.0,0.0,6.0,6.0,12.0
223,8927583,4,4,0,0.0,4.0,0.0,0.0,8,4.0,4.0,0.0,4.0,8.0


In [130]:
df_check[(df_check.Total_Extracted == df_check.Total_Extracted) & (df_check.present != df_check.prophetic)].sort_values("prophetic_error",ascending=False)

,patentnumber,present,past,unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
207,8927555,51,53,0,0.0,104.0,0.0,0.0,104,104.0,51.0,51.0,0.0,102.0
581,8933104,43,75,0,18.0,100.0,0.0,1.0,118,118.0,25.0,25.0,0.0,50.0
205,8927550,1,22,0,22.0,1.0,0.0,1.0,23,23.0,21.0,21.0,0.0,42.0
577,8933098,4,25,0,24.0,5.0,0.0,1.0,29,29.0,20.0,20.0,0.0,40.0
321,8927757,17,21,0,37.0,1.0,0.0,1.0,38,38.0,20.0,20.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,8927795,1,0,0,0.0,1.0,0.0,0.0,1,1.0,1.0,1.0,0.0,2.0
330,8927775,0,22,0,1.0,21.0,0.0,1.0,22,22.0,1.0,1.0,0.0,2.0
30,8926736,6,27,0,7.0,26.0,0.0,1.0,33,33.0,1.0,1.0,0.0,2.0
313,8927744,2,1,0,1.0,2.0,0.0,1.0,3,3.0,1.0,1.0,0.0,2.0


In [132]:
df_check[(df_check.Total_Extracted == df_check.Total_Extracted) & (df_check.past != df_check.nonprophetic)].sort_values("nonprophetic_error",ascending=False).head(20)

,patentnumber,present,past,unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
558,8933072,4,61,0,0.0,9.0,0.0,0.0,65,9.0,4.0,52.0,56.0,112.0
207,8927555,51,53,0,0.0,104.0,0.0,0.0,104,104.0,51.0,51.0,0.0,102.0
581,8933104,43,75,0,18.0,100.0,0.0,1.0,118,118.0,25.0,25.0,0.0,50.0
205,8927550,1,22,0,22.0,1.0,0.0,1.0,23,23.0,21.0,21.0,0.0,42.0
321,8927757,17,21,0,37.0,1.0,0.0,1.0,38,38.0,20.0,20.0,0.0,40.0
577,8933098,4,25,0,24.0,5.0,0.0,1.0,29,29.0,20.0,20.0,0.0,40.0
195,8927533,17,32,0,0.0,49.0,0.0,0.0,49,49.0,17.0,17.0,0.0,34.0
182,8927513,32,33,0,49.0,16.0,0.0,1.0,65,65.0,17.0,17.0,0.0,34.0
286,8927698,7,20,0,23.0,3.0,0.0,1.0,27,26.0,16.0,17.0,1.0,34.0
421,8932558,38,26,0,55.0,9.0,0.0,1.0,64,64.0,17.0,17.0,0.0,34.0


In [69]:
soup.find_all()

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE us-patent-grant SYSTEM "us-patent-grant-v45-2014-04-03.dtd">
<us-patent-grant country="US" date-produced="20141219" date-publ="20150106" dtd-version="v4.5 2014-04-03" file="US08927710-20150106.XML" id="us-patent-grant" lang="EN" status="PRODUCTION">
<us-bibliographic-data-grant>
<publication-reference>
<document-id>
<country>US</country>
<doc-number>08927710</doc-number>
<kind>B2</kind>
<date>20150106</date>
</document-id>
</publication-reference>
<application-reference appl-type="utility">
<document-id>
<country>US</country>
<doc-number>13378334</doc-number>
<date>20100614</date>
</document-id>
</application-reference>
<us-application-series-code>13</us-application-series-code>
<priority-claims>
<priority-claim kind="national" sequence="01">
<country>JP</country>
<doc-number>2009-142166</doc-number>
<date>20090615</date>
</priority-claim>
</priority-claims>
<us-term-of-grant>
<us-term-extension>325</us-term-extension>
</us-term-of-grant

In [119]:
from bs4 import BeautifulSoup
xml = merged[merged.patentnumber == "8933222"].xml.values[0]
soup = BeautifulSoup(xml, "xml")
exammples = extract_examples_start_w_word_test(soup.find_all(["heading","p"]))

In [121]:
ex = "(S)-3-(4′-Cyano-biphenyl-4-yl)-2-{[(S)-3-[4-(3,4-dichloro-benzyloxy)-phenyl]-2-oxo-6-((R)-tetrahydro-furan-2-carbonyl)-2,3,5,6,7,8-hexahydro-1H-4-oxa-1,6-diaza-anthracene-7-carbonyl]-amino}-propionic acid Title compound (62 mg) was prepared from Intermediate A (76 mg, 0.1 mmol) following general procedures F and B. LC-MS (m/z): 845."
analyze_sentence_tense(ex)

'present'

In [128]:
for example in exammples:
    text = example["title"] + " ".join(example["content"])
    tense = analyze_sentence_tense(text)
    if tense == "present":
        print(text)


In [122]:
stats["documents_with_examples"]["8927710"]

[{'number': 'Example 1',
  'title': '',
  'content': ['',
   'A dichloromethane (90 mL) solution of compound 1A (12.8 g, 89.4 mmol) and pyridine (8.50 g, 107 mmol) was cooled to 1 to 3° C., and a dichloromethane (90 mL) solution of benzyloxyacetyl chloride (19.8 g, 107 mmol) was added dropwise over 50 minutes while the same temperature was retained. After the reaction solution was stirred at the same temperature for 30 minutes, temperature was gradually raised to 15° C. over 60 minutes, and ice water was added. The dichloromethane layer was separated, and the aqueous layer was extracted with dichloromethane once. The combined extracts were washed with water three times, washed with an aqueous saturated sodium chloride solution, and dried. The solvent was distilled off, and the resulting oil was purified by silica gel column chromatography. The materials were eluted firstly with n-hexane and, then, with n-hexane-ethyl acetate (1:1, v/v). Concentration of objective fraction afforded 22.2

In [134]:
from utilities.database_utils import *

In [137]:
store_patent_examples(stats["documents_with_examples"])
store_patent_statistics(final)

In [177]:
print(f"we extract more than janets: {len(df_check[df_check.Total_Extracted > df_check.Total_Freilich])}, total difference: {df_check[df_check.Total_Extracted > df_check.Total_Freilich].Total_Extracted.sum() - df_check[df_check.Total_Extracted > df_check.Total_Freilich].Total_Freilich.sum()}")
print(f"we extract less than janets: {len(df_check[df_check.Total_Extracted < df_check.Total_Freilich])}, total difference: {df_check[df_check.Total_Extracted < df_check.Total_Freilich].Total_Freilich.sum() - df_check[df_check.Total_Extracted < df_check.Total_Freilich].Total_Extracted.sum()}")
print(f"we extract equal to janets: {len(df_check[df_check.Total_Extracted == df_check.Total_Freilich])}")

we extract more than janets: 97, total difference: 256.0
we extract less than janets: 3, total difference: 3.0
we extract equal to janets: 699


In [178]:
df_check[df_check.Total_Extracted < df_check.Total_Freilich]

,patentnumber,present,past,unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
87,8927015,0,24,0,1.0,24.0,0.0,1.0,24,25.0,1.0,0.0,1.0,2.0
199,8927541,3,17,0,20.0,1.0,0.0,1.0,20,21.0,17.0,16.0,1.0,34.0
444,8932615,2,0,0,1.0,2.0,0.0,1.0,2,3.0,1.0,2.0,1.0,4.0
